In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Advanced Interior Reimagining AI - Google Colab Runner\n",
    "\n",
    "This notebook sets up and runs the Advanced Interior Reimagining AI application in Google Colab."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 1: Install Required Dependencies"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "!pip install torch torchvision diffusers transformers gradio opencv-python Pillow accelerate xformers safetensors huggingface-hub sentencepiece"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 2: Set Up Project Structure"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "import os\n",
    "\n",
    "# Create project directories\n",
    "os.makedirs('models', exist_ok=True)\n",
    "os.makedirs('ui', exist_ok=True)\n",
    "os.makedirs('utils', exist_ok=True)\n",
    "\n",
    "print(\"Project directories created!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 3: Clone the Repository Files\n",
    "\n",
    "Let's download the code files from the repository."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Clone the repository files directly into the created structure\n",
    "!git clone https://github.com/yourusername/interior-reimagining-ai.git temp_repo\n",
    "\n",
    "# Move files to the right locations\n",
    "!cp -r temp_repo/models/* models/\n",
    "!cp -r temp_repo/ui/* ui/\n",
    "!cp -r temp_repo/utils/* utils/\n",
    "!cp temp_repo/main.py .\n",
    "\n",
    "# Clean up\n",
    "!rm -rf temp_repo\n",
    "\n",
    "print(\"Repository files downloaded and organized!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Alternative: Create Files Directly\n",
    "\n",
    "If the repository cloning doesn't work, we can create the necessary files directly."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Create __init__.py files\n",
    "with open('models/__init__.py', 'w') as f:\n",
    "    f.write(\"\"\"\n",
    "from .image_processor import ImageProcessor, ProcessedImage, RoomSegmentClass\n",
    "from .interior_reimaginer import InteriorReimaginer\n",
    "from .design_styles import DesignStyle, load_design_styles\n",
    "\n",
    "__all__ = [\n",
    "    'ImageProcessor', \n",
    "    'ProcessedImage', \n",
    "    'RoomSegmentClass', \n",
    "    'InteriorReimaginer', \n",
    "    'DesignStyle', \n",
    "    'load_design_styles'\n",
    "]\n",
    "\"\"\")\n",
    "\n",
    "with open('ui/__init__.py', 'w') as f:\n",
    "    f.write(\"\"\"\n",
    "from .gradio_interface import create_advanced_ui\n",
    "\n",
    "__all__ = ['create_advanced_ui']\n",
    "\"\"\")\n",
    "\n",
    "with open('utils/__init__.py', 'w') as f:\n",
    "    f.write(\"\"\"\n",
    "from .helpers import (\n",
    "    setup_logging,\n",
    "    get_device,\n",
    "    print_system_info,\n",
    "    resize_for_model,\n",
    "    save_temp_image,\n",
    "    create_visualization\n",
    ")\n",
    "\n",
    "__all__ = [\n",
    "    'setup_logging',\n",
    "    'get_device',\n",
    "    'print_system_info',\n",
    "    'resize_for_model',\n",
    "    'save_temp_image',\n",
    "    'create_visualization'\n",
    "]\n",
    "\"\"\")\n",
    "\n",
    "print(\"__init__.py files created!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 4: Create a Simple Main Script for Colab\n",
    "\n",
    "Let's create a simplified main script that will work in Colab."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "%%writefile colab_main.py\n",
    "import os\n",
    "import logging\n",
    "import torch\n",
    "\n",
    "from models import InteriorReimaginer\n",
    "from ui import create_advanced_ui\n",
    "from utils import setup_logging, print_system_info\n",
    "\n",
    "# Configure logging\n",
    "setup_logging(debug=True)\n",
    "logger = logging.getLogger(__name__)\n",
    "\n",
    "# Determine device\n",
    "if torch.cuda.is_available():\n",
    "    device = \"cuda\"\n",
    "    logger.info(f\"Using CUDA device: {torch.cuda.get_device_name(0)}\")\n",
    "else:\n",
    "    device = \"cpu\"\n",
    "    logger.warning(\"No GPU detected, using CPU. Processing will be significantly slower.\")\n",
    "\n",
    "# Print system info\n",
    "print_system_info()\n",
    "\n",
    "# Create the reimaginer\n",
    "model_id = \"stabilityai/stable-diffusion-xl-refiner-1.0\"\n",
    "logger.info(f\"Initializing Interior Reimaginer with model {model_id}...\")\n",
    "reimaginer = InteriorReimaginer(base_model_id=model_id, device=device)\n",
    "\n",
    "# Create and launch UI\n",
    "logger.info(\"Starting web interface...\")\n",
    "ui = create_advanced_ui(reimaginer)\n",
    "\n",
    "# Launch with a public link for Colab access\n",
    "ui.launch(share=True)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 5: Enable System Memory for Colab\n",
    "\n",
    "Let's enable swap space to prevent out-of-memory errors."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Create 8GB of swap space to prevent OOM issues\n",
    "!sudo fallocate -l 8G /swapfile\n",
    "!sudo chmod 600 /swapfile\n",
    "!sudo mkswap /swapfile\n",
    "!sudo swapon /swapfile\n",
    "!free -h"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 6: Check System and GPU Information"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "import torch\n",
    "\n",
    "print(f\"PyTorch version: {torch.__version__}\")\n",
    "print(f\"CUDA available: {torch.cuda.is_available()}\")\n",
    "\n",
    "if torch.cuda.is_available():\n",
    "    print(f\"CUDA device: {torch.cuda.get_device_name(0)}\")\n",
    "    print(f\"CUDA device count: {torch.cuda.device_count()}\")\n",
    "    !nvidia-smi\n",
    "else:\n",
    "    print(\"No GPU detected, running on CPU (this will be significantly slower)\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 7: Download All Required Models\n",
    "\n",
    "Pre-download all the models to avoid timeout issues during application startup."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Pre-download all required models\n",
    "from huggingface_hub import snapshot_download\n",
    "\n",
    "# Download SDXL model\n",
    "print(\"Downloading Stable Diffusion XL base model...\")\n",
    "snapshot_download(repo_id=\"stabilityai/stable-diffusion-xl-base-1.0\")\n",
    "\n",
    "print(\"Downloading Stable Diffusion XL refiner model...\")\n",
    "snapshot_download(repo_id=\"stabilityai/stable-diffusion-xl-refiner-1.0\")\n",
    "\n",
    "# Download ControlNet models\n",
    "print(\"Downloading ControlNet models...\")\n",
    "snapshot_download(repo_id=\"diffusers/controlnet-depth-sdxl-1.0-small\")\n",
    "snapshot_download(repo_id=\"diffusers/controlnet-canny-sdxl-1.0\")\n",
    "\n",
    "# Download other models\n",
    "print(\"Downloading depth estimation model...\")\n",
    "snapshot_download(repo_id=\"Intel/dpt-large\")\n",
    "\n",
    "print(\"Downloading segmentation model...\")\n",
    "snapshot_download(repo_id=\"nvidia/segformer-b5-finetuned-ade-640-640\")\n",
    "\n",
    "print(\"Downloading CLIP segmentation model...\")\n",
    "snapshot_download(repo_id=\"CIDAS/clipseg-rd64-refined\")\n",
    "\n",
    "print(\"Downloading image captioning model...\")\n",
    "snapshot_download(repo_id=\"Salesforce/blip-image-captioning-large\")\n",
    "\n",
    "print(\"All models downloaded!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 8: Run the Application"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Run the application\n",
    "!python colab_main.py"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Alternative Run Method\n",
    "\n",
    "If the above approach doesn't work, you can also try running the Gradio interface directly in the notebook:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "import os\n",
    "import logging\n",
    "import torch\n",
    "\n",
    "from models import InteriorReimaginer\n",
    "from ui import create_advanced_ui\n",
    "from utils import setup_logging, print_system_info\n",
    "\n",
    "# Configure logging\n",
    "setup_logging(debug=True)\n",
    "logger = logging.getLogger(__name__)\n",
    "\n",
    "# Determine device\n",
    "if torch.cuda.is_available():\n",
    "    device = \"cuda\"\n",
    "    logger.info(f\"Using CUDA device: {torch.cuda.get_device_name(0)}\")\n",
    "else:\n",
    "    device = \"cpu\"\n",
    "    logger.warning(\"No GPU detected, using CPU. Processing will be significantly slower.\")\n",
    "\n",
    "# Create the reimaginer\n",
    "logger.info(\"Initializing Interior Reimaginer...\")\n",
    "reimaginer = InteriorReimaginer(device=device)\n",
    "\n",
    "# Create and launch UI\n",
    "logger.info(\"Starting web interface...\")\n",
    "ui = create_advanced_ui(reimaginer)\n",
    "ui.launch(share=True, debug=True)"
   ]
  }
 ],
 "metadata": {
  "accelerator": "GPU",
  "colab": {
   "gpuType": "T4",
   "provenance": []
  },
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}